In [161]:
# !pip install bs4 --user

In [162]:
# Import dependencies
import pandas as pd
import pymongo
import requests
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [163]:
# Configure ChromeDriver
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dartanion\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


# NASA Mars News

In [164]:
def mars_news():
    url = 'https://mars.nasa.gov/news/'
    browser.visit(url)

    html = browser.html
    news_soup = BeautifulSoup(html, 'html.parser')

    article_container = news_soup.find('ul', class_='item_list')

    headline_date = article_container.find('div', class_='list_date').text
    news_title = article_container.find('div', class_='content_title').find('a').text
    news_p = article_container.find('div', class_='article_teaser_body').text

    return news_title, news_p

# JPL Mars Space Images - Featured Image




In [165]:
def featured_image():
    base_url = 'https://www.jpl.nasa.gov'

    url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(url)

    html = browser.html
    img_soup = BeautifulSoup(html, 'html.parser')

    # Method 1: parsing through the style attribute in the article tag
    try:
        img_elem = img_soup.find('article', class_='carousel_item')['style']
        img_rel_url = img_elem.replace("background-image: url('", '')
        img_rel_url = img_rel_url.replace("');", '')
        print(img_rel_url)
    except Exception as e:
        print(e)

    # Method 2: clicking the FULL TEXT button and pulling the image
    try:
        full_image_elem = browser.find_by_id('full_image')[0]
        full_image_elem.click()

        html = browser.html
        img_soup = BeautifulSoup(html, 'html.parser')

        img_rel_url = img_soup.find('img', class_='fancybox-image')['src']
        print(img_rel_url)
    except Exception as e:
        print(e)

    featured_image_url  = f'{base_url}{img_rel_url}'
    print(featured_image_url)
    
    return featured_image_url

# Mars Facts

In [166]:
def mars_facts():
    url = 'https://space-facts.com/mars/'
    browser.visit(url)

    mars_facts_df = pd.read_html(url)
    mars_facts_df = mars_facts_df[0]
    mars_facts_df.columns = ['Description', 'Mars']
    mars_facts_df

    mars_facts_html = mars_facts_df.to_html(classes='table table-striped')
    
    return mars_facts_html

In [167]:
# if you wanted to pull the HTML table directly from the page
# w/o Pandas, then you can. The HTML would, however, contain
# the formatting from the original web developer

html = browser.html
facts_soup = BeautifulSoup(html, 'html.parser')
facts_soup.find(id='tablepress-p-mars')

# Mars Hemispheres

This site appears to be down as of 8:15 pm on 12/29. We will revisit this section.

In [168]:
# Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.


# You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.


# Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.


# Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.


# # Example:
# hemisphere_image_urls = [
#     {"title": "Valles Marineris Hemisphere", "img_url": "..."},
#     {"title": "Cerberus Hemisphere", "img_url": "..."},
#     {"title": "Schiaparelli Hemisphere", "img_url": "..."},
#     {"title": "Syrtis Major Hemisphere", "img_url": "..."},
# ]

# Insert into Mongo DB

In [169]:
def scrape_all():

    # Populate variables from the functions
    news_title, news_p = mars_news()
    featured_img_url = featured_image()
    mars_facts_html = mars_facts()

    # Assemble the document to insert into the database
    nasa_document = {
        'news_title': news_title,
        'news_paragraph': news_p,
        'featured_img_url': featured_img_url,
        'mars_facts_html': mars_facts_html
    }

    return nasa_document

In [170]:
# Connect to MongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Connect to mars_app database
db = client.mars_app

# Connect to mars collection
mars = db.mars

# Gather document to insert
data_document = scrape_all()

# Insert into the mars collection
# mars.insert_one(data_document)

# Upsert into the mars collection (preferred to avoid duplicates)
mars.update_one({}, {'$set': data_document}, upsert=True)

/spaceimages/images/wallpaper/PIA14872-1920x1200.jpg
/spaceimages/images/mediumsize/PIA14872_ip.jpg
https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA14872_ip.jpg
